In [57]:
import numpy as np
import os
from scipy.io import wavfile
import librosa
import csv
import math as m
from natsort import natsorted
from sklearn import preprocessing
import h5py
import soundfile as sf
#in this section the features for mixed voices are extracted.
# orig_path=os.getcwd()
i = 1
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/10hdata'
mixed_folder = os.path.normpath(os.path.join(write_path,'mixed_10h'))
# f.close()
# a = os.listdir(os.path.join(mixed_folder))
# a = natsorted(a)
# mixed_logname = 'mixed_log_10h_norm'
# mixed_phasename = 'mixed_phase_10h_norm'
# mixed_logpath = os.path.normpath(os.path.join(write_path,mixed_logname))
# mixed_phasepath = os.path.normpath(os.path.join(write_path,mixed_phasename))

# f = h5py.File(mixed_logpath+'.hdf5', 'w')
# d = f.create_dataset(mixed_logname, (0,257),maxshape=(None,257), dtype='f', chunks=True)
# # ff.close()
# ff = h5py.File(mixed_phasepath+'.hdf5', 'w')
# dd = ff.create_dataset(mixed_phasename, (0,257),maxshape=(None,257), dtype='f', chunks=True)
# #################################
# w=3
# # #################################
# for filename in a:
#     wav_data, sr = librosa.load(os.path.join(mixed_folder,filename), sr=16000) 
#     framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
#     print(framed_data.shape)
#     abslt=np.absolute(framed_data)**2
#     dft_signal=np.log10(abslt+1e-7*np.ones(np.shape(abslt)))
#     for i in range(m.floor(w/2)):
#         dft_signal=np.insert(dft_signal,0,0,axis = 1)
#     data_phase=np.angle(framed_data)
#     for i in range(m.floor(w/2)):
#         data_phase=np.insert(data_phase,0,0,axis = 1)
#     d.resize(d.shape[0]+dft_signal.shape[1], axis=0)   
#     d[-1*dft_signal.shape[1]:] = dft_signal.T
#     dd.resize(dd.shape[0]+data_phase.shape[1], axis=0)   
#     dd[-1*data_phase.shape[1]:] = data_phase.T
#     i=i+1
# f.close()
# ff.close()
###################################################
# a = os.listdir(os.path.join(mixed_folder))
# a = natsorted(a)
# mixed_logname = 'mixed_log_10h_nozeroinsert'
# mixed_phasename = 'mixed_phase_10h_nozeroinsert'
# mixed_logpath = os.path.normpath(os.path.join(write_path,mixed_logname))
# mixed_phasepath = os.path.normpath(os.path.join(write_path,mixed_phasename))

# f = h5py.File(mixed_logpath+'.hdf5', 'w')
# d = f.create_dataset(mixed_logname, (0,257),maxshape=(None,257), dtype='f', chunks=True)
# # ff.close()
# ff = h5py.File(mixed_phasepath+'.hdf5', 'w')
# dd = ff.create_dataset(mixed_phasename, (0,257),maxshape=(None,257), dtype='f', chunks=True)
# # #################################
# for filename in a:
#     wav_data, sr = librosa.load(os.path.join(mixed_folder,filename), sr=16000) 
#     framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
# #     print(framed_data.shape)
#     abslt=np.absolute(framed_data)**2
#     dft_signal=np.log10(abslt+1e-7*np.ones(np.shape(abslt)))
# #     for i in range(m.floor(w/2)):
# #         dft_signal=np.insert(dft_signal,0,0,axis = 1)
#     data_phase=np.angle(framed_data)
# #     for i in range(m.floor(w/2)):
# #         data_phase=np.insert(data_phase,0,0,axis = 1)
#     d.resize(d.shape[0]+dft_signal.shape[1], axis=0)   
#     d[-1*dft_signal.shape[1]:] = dft_signal.T
#     dd.resize(dd.shape[0]+data_phase.shape[1], axis=0)   
#     dd[-1*data_phase.shape[1]:] = data_phase.T
#     i=i+1
# f.close()
# ff.close()
# print('finished')
###################################################
# import h5py
# import math as m
# import os
# import numpy as np
# import librosa
# from natsort import natsorted
# orig_path=os.getcwd()
# wav=[]
# clean_raw = 'clean_10h'
# l=os.listdir(os.path.normpath(os.path.join(write_path,clean_raw)))
# l=natsorted(l)
# filename1 = l[1]
# i=0
# ###################################################
# w=3
# ###################################################
# num_of_snrs=6
# num_of_speakers=6
# # files = os.listdir(os.path.normpath(os.path.join(Data_path,clean_raw,filename1)))
# files = os.listdir(os.path.normpath(os.path.join(write_path,clean_raw)))
# files=natsorted(files)
# # f.close()
# # ff.close()
# clean_log = 'single_dataset_log_10h_norm'
# clean_phase = 'single_dataset_phase_10h_norm'
# clean_log_path = os.path.normpath(os.path.join(write_path,clean_log))
# clean_phase_path = os.path.normpath(os.path.join(write_path,clean_phase))
# f = h5py.File(clean_log_path+'.hdf5', 'w')
# d = f.create_dataset(clean_log, (0,257),maxshape=(None,257), dtype='f', chunks=True)
# ff = h5py.File(clean_phase_path+'.hdf5', 'w')
# dd = ff.create_dataset(clean_phase, (0,257),maxshape=(None,257), dtype='f', chunks=True)
# #in this section the features for clean voices are extracted.
# for voice in files:
#     wav_data, sr = librosa.load(os.path.join(write_path,clean_raw,voice), sr=16000) 
#     framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
#     print(framed_data.shape)
#     abslt=np.absolute(framed_data)**2
#     dft_signal=np.log10(abslt+1e-7*np.ones(np.shape(abslt)))
#     data_phase=np.angle(framed_data)
# #     for i in range(m.floor(w/2)):
# #         dft_signal=np.insert(dft_signal,0,0,axis = 1)
# #     for i in range(num_of_snrs*num_of_speakers):
#     d.resize(d.shape[0]+dft_signal.shape[1], axis=0)   
#     d[-1*dft_signal.shape[1]:] = dft_signal.T
#     dd.resize(dd.shape[0]+data_phase.shape[1], axis=0)   
#     dd[-1*data_phase.shape[1]:] = data_phase.T
# ##############################################
# ##############################################
# import math as m
# import numpy as np
# import h5py
# ##############################################
# w = 3
# ###############################################
# mixed_logname='mixed_log_10h'
# mixed_phasename='mixed_phase_10h'
# mixed_log_path = os.path.normpath(os.path.join(write_path,mixed_logname))
# clean_phase_path = os.path.normpath(os.path.join(write_path,mixed_phasename))
# import h5py
# h5f = h5py.File(mixed_log_path+'.hdf5','r')
# ftr = h5f[mixed_logname][0:]
# # h5f.close()
# ftr_refrmd=[]
# ftr_refrmd_norm=[]
# print('0')
# for i in range(m.floor(w/2)):
#     p=-m.floor(w/2)
#     temp = []
#     for j in range(w):
#         if np.all(ftr[i]==0):
#             break
#         temp.extend(ftr[i+p+j])
#     if np.any(temp!=[]):
#         ftr_refrmd.append(temp)
# # f.close()
# print('1')
# refrmd_file='ftr_refrmd_10h_norm'
# refrmd_file_path = os.path.normpath(os.path.join(write_path,refrmd_file))
# f = h5py.File(refrmd_file_path+'.hdf5', 'w')
# d = f.create_dataset(refrmd_file, (0,257*w),maxshape=(None,257*w), dtype='f', chunks=True)
# print('2')
# for i in range(m.floor(w/2),ftr.shape[0]-m.floor(w/2)-1):
#     k=-m.floor(w/2)
#     temp = []
#     for j in range(w):
#         if np.all(ftr[i]==0):
#             break
#         temp.extend(ftr[i+k])
#         k=k+1
#     if np.any(temp!=[]):
#         ftr_refrmd.append(temp)
#     if len(ftr_refrmd)>50000 or i==ftr.shape[0]-m.floor(w/2)-2:
#         print(len(ftr_refrmd))
# #         for ftr_num in range(len(ftr_refrmd)):
#         ftr_refrmd_norm = preprocessing.scale(ftr_refrmd, axis=0, with_mean=True, with_std=True, copy=True)
# #             print(ftr_refrmd_norm.shape)
#         d.resize(d.shape[0]+len(ftr_refrmd_norm), axis=0)   
#         d[-1*len(ftr_refrmd_norm):] = ftr_refrmd_norm
#         print(len(ftr_refrmd_norm))
#         ftr_refrmd_norm=[]
#         ftr_refrmd=[]
# #     if i%10000==0:
# #         print(i)
# for i in range(ftr.shape[0]-m.floor(w/2)-1,ftr.shape[0]):
#     p=-m.floor(w/2)
#     temp = []
#     for j in range(w):
#         if np.all(ftr[i]==0):
#             break
#         if j>m.floor(w/2)+(ftr.shape[0]-i)-1:
#             temp.extend(np.zeros(ftr[ftr.shape[0]-1].shape))
#         else:
#             temp.extend(ftr[i+p+j])
#     if np.any(temp!=[]):
#         ftr_refrmd.append(temp)
# # for ftr_num in range(len(ftr_refrmd)):
# ftr_refrmd_norm = preprocessing.scale(ftr_refrmd, axis=0, with_mean=True, with_std=True, copy=True)
# d.resize(d.shape[0]+len(ftr_refrmd_norm), axis=0)   
# d[-1*len(ftr_refrmd_norm):] = ftr_refrmd_norm
# ftr_refrmd=[]
# ftr_refrmd_norm=[]
# f.close()
# h5f.close()
# ##############################################
# ##############################################
import h5py
import numpy as np
file='mixed_log_10h_nozeroinsert'
file_path = os.path.normpath(os.path.join(write_path,file))
h5f = h5py.File(file_path+'.hdf5', 'r')
d=h5f[file]
tot_len=d.shape[0]
dim = d.shape[1]
lst=0
tot_num=(tot_len//10000)*10000
for i in range(0,tot_num,10000):
    print(i)
    mixed = h5f[file][i:i+10000]
    lst=lst+np.sum(mixed,axis=0)

mixed = h5f[file][tot_num:]
lst=lst+np.sum(mixed,axis=0)
# print(mixed.shape)
tot_mean=lst/tot_len
lst=0
h5f = h5py.File(file_path+'.hdf5','r')
for i in range(0,tot_num,10000):
    mixed = h5f[file][i:i+10000]
    lst=lst+np.sum((mixed - tot_mean)**2,axis=0)
del mixed  
mixed = h5f[file][tot_num:]
lst=lst+np.sum((mixed - tot_mean)**2,axis=0)
lst=lst/tot_len
tot_std=np.sqrt(lst)
h5f.close()
normalized_file = 'mixed_log_10h_norm2'
normalized_file_path =  os.path.normpath(os.path.join(write_path,normalized_file))
f = h5py.File(normalized_file_path+'.hdf5', 'w')
dd = f.create_dataset(normalized_file, (0,dim),maxshape=(None,dim), dtype='f', chunks=True)
h5f = h5py.File(file_path+'.hdf5','r')
for i in range(0,tot_num,10000):
    print(i)
    mixed = h5f[file][i:i+10000]
    scaled=(mixed-tot_mean)/tot_std
    dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
    dd[-1*scaled.shape[0]:] = scaled
del mixed    
mixed = h5f[file][tot_num:]
# print(mixed.shape)
scaled=(mixed-tot_mean)/tot_std
dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
dd[-1*scaled.shape[0]:] = scaled

np.savetxt(os.path.normpath(os.path.join(write_path,'mean_mixed_log.txt')),tot_mean)
np.savetxt(os.path.normpath(os.path.join(write_path,'std_mixed_log.txt')),tot_std)
############################################
############################################
# import h5py
# import numpy as np
# refrmd_file='ftr_refrmd_10h'
# refrmd_file_path = os.path.normpath(os.path.join(write_path,refrmd_file))
# h5f = h5py.File(refrmd_file_path+'.hdf5', 'r')
# d=h5f[refrmd_file]
# tot_len=d.shape[0]
# dim = d.shape[1]
# lst=0

# for i in range(0,tot_num,10000):
#     print(i)
#     mixed = h5f[refrmd_file][i:i+10000]
#     lst=lst+np.sum(mixed,axis=0)
# tot_num=(tot_len//10000)*10000
# mixed = h5f[refrmd_file][tot_num:]
# lst=lst+np.sum(mixed,axis=0)
# print(mixed.shape)
# tot_mean=lst/tot_len
# lst=0
# h5f = h5py.File(refrmd_file_path+'.hdf5','r')
# for i in range(0,tot_num,10000):
#     mixed = h5f[refrmd_file][i:i+10000]
#     lst=lst+np.sum((mixed - tot_mean)**2,axis=0)
# del mixed  
# mixed = h5f[refrmd_file][tot_num:]
# lst=lst+np.sum((mixed - tot_mean)**2,axis=0)
# lst=lst/tot_len
# tot_std=np.sqrt(lst)
# h5f.close()
# normalized_file = 'ftr_refrmd_10h_norm2'
# normalized_file_path =  os.path.normpath(os.path.join(write_path,normalized_file))
# f = h5py.File(normalized_file_path+'.hdf5', 'w')
# dd = f.create_dataset(normalized_file, (0,dim),maxshape=(None,dim), dtype='f', chunks=True)
# h5f = h5py.File(refrmd_file_path+'.hdf5','r')
# for i in range(0,tot_num,10000):
#     print(i)
#     mixed = h5f[refrmd_file][i:i+10000]
#     scaled=(mixed-tot_mean)/tot_std
#     dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
#     dd[-1*scaled.shape[0]:] = scaled
# del mixed    
# mixed = h5f[refrmd_file][tot_num:]
# print(mixed.shape)
# scaled=(mixed-tot_mean)/tot_std
# dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
# dd[-1*scaled.shape[0]:] = scaled

# np.savetxt(os.path.normpath(os.path.join(write_path,refrmd_file,'mean.txt')),tot_mean)
# np.savetxt(os.path.normpath(os.path.join(write_path,refrmd_file,'std.txt')),tot_std)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\studies\\university\\thesis\\speech_separation_codes\\du16\\donesomestuff\\10hdata\\mixed_log_10h_nozeroinsert\\mean_mixed_log.txt'

In [97]:
np.savetxt(os.path.normpath(os.path.join(write_path,'mean.txt')),tot_mean)
np.savetxt(os.path.normpath(os.path.join(write_path,'std.txt')),tot_std)

In [42]:
wav_data, sr = librosa.load(os.path.join(mixed_folder,filename), 16000) 
import sounddevice as sd
sd.play(wav_data, 16000)
sr

16000

In [58]:
np.savetxt(os.path.normpath(os.path.join(write_path,'mean_mixed_log.txt')),tot_mean)
np.savetxt(os.path.normpath(os.path.join(write_path,'std_mixed_log.txt')),tot_std)

In [54]:
f.close()
ff.close()

In [46]:
def reconstruct(wave,angle):
    recon = np.sqrt(np.power(10, wave))
    recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=256, win_length=512, window='hann')
    return recon
recon = reconstruct(dft_signal.T, data_phase.T)
sd.play(recon, 16000)
sf.write('D:\\studies\\university\\thesis\\speech_separation_codes\\du16\\donesomestuff\\trash\\1.wav',recon,16000)

TypeError: write() missing 1 required positional argument: 'samplerate'

In [34]:
data_phase.shape

(257, 126)

In [35]:
tot_num=(tot_len//10000)*10000
mixed2 = h5f[normalized_file]
mixed2.shape

NameError: name 'tot_len' is not defined

In [94]:
f.close()
h5f.close()

In [30]:
a = [[10,2,100,50,46,365],[20,35,10,35,211,5]]
at = preprocessing.scale(a, axis=0, with_mean=True, with_std=True, copy=True)
at
np.std(at[1])

1.0

In [3]:
import sounddevice as sd
sd.play(wav_data)

In [60]:
import numpy as np
import tensorflow as tf
import os
__author__ = "Sangwoong Yoon"

def np_to_tfrecords(X, Y, file_path_prefix, verbose=True):
    """
    Converts a Numpy array (or two Numpy arrays) into a tfrecord file.
    For supervised learning, feed training inputs to X and training labels to Y.
    For unsupervised learning, only feed training inputs to X, and feed None to Y.
    The length of the first dimensions of X and Y should be the number of samples.
    
    Parameters
    ----------
    X : numpy.ndarray of rank 2
        Numpy array for training inputs. Its dtype should be float32, float64, or int64.
        If X has a higher rank, it should be rshape before fed to this function.
    Y : numpy.ndarray of rank 2 or None
        Numpy array for training labels. Its dtype should be float32, float64, or int64.
        None if there is no label array.
    file_path_prefix : str
        The path and name of the resulting tfrecord file to be generated, without '.tfrecords'
    verbose : bool
        If true, progress is reported.
    
    Raises
    ------
    ValueError
        If input type is not float (64 or 32) or int.
    
    """
    def _dtype_feature(ndarray):
        """match appropriate tf.train.Feature class with dtype of ndarray. """
        assert isinstance(ndarray, np.ndarray)
        dtype_ = ndarray.dtype
        if dtype_ == np.float64 or dtype_ == np.float32:
            return lambda array: tf.train.Feature(float_list=tf.train.FloatList(value=array))
        elif dtype_ == np.int64:
            return lambda array: tf.train.Feature(int64_list=tf.train.Int64List(value=array))
        else:  
            raise ValueError("The input should be numpy ndarray. \
                               Instaed got {}".format(ndarray.dtype))
            
    assert isinstance(X, np.ndarray)
    assert len(X.shape) == 2  # If X has a higher rank, 
                               # it should be rshape before fed to this function.
    assert isinstance(Y, np.ndarray) or Y is None
    
    # load appropriate tf.train.Feature class depending on dtype
    dtype_feature_x = _dtype_feature(X)
    if Y is not None:
        assert X.shape[0] == Y.shape[0]
        assert len(Y.shape) == 2
        dtype_feature_y = _dtype_feature(Y)            
    
    # Generate tfrecord writer
    result_tf_file = file_path_prefix + '.tfrecords'
    writer = tf.python_io.TFRecordWriter(result_tf_file)
    if verbose:
        print("Serializing {:d} examples into {}".format(X.shape[0], result_tf_file))
        
    # iterate over each sample,
    # and serialize it as ProtoBuf.
    for idx in range(X.shape[0]):
        x = X[idx]
        if Y is not None:
            y = Y[idx]
        
        d_feature = {}
        d_feature['X'] = dtype_feature_x(x)
        if Y is not None:
            d_feature['Y'] = dtype_feature_y(y)
            
        features = tf.train.Features(feature=d_feature)
        example = tf.train.Example(features=features)
        serialized = example.SerializeToString()
        writer.write(serialized)
    
    if verbose:
        print("Writing {} done!".format(result_tf_file))

        
#################################    
##      Test and Use Cases     ##
#################################
import h5py
load_size = 100000
k = 0
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/10hdata'
input_name = 'mixed_log_10h_norm2'
input_path = os.path.normpath(os.path.join(write_path,input_name))
target_name = 'single_dataset_log_10h'
target_path = os.path.normpath(os.path.join(write_path,target_name))
h5f1 = h5py.File(input_path+'.hdf5','r')
hh = h5py.File(target_path+'.hdf5', 'r')
d=hh[target_name]
data_len = d.shape[0]
index = np.arange(0, data_len, load_size)
for I in index:
    if data_len-I<load_size:
        xx = h5f1[input_name][I:]
        yy = hh[target_name][I:]
    else:
        xx = h5f1[input_name][I:I+load_size]
        yy = hh[target_name][I:I+load_size]
    np_to_tfrecords(xx, yy, os.path.normpath(os.path.join(Data_path,'tfrecord_files','mixed_10h_norm2','filenum'+str(k))), verbose=True)
    k = k+1
# 1-2. Check if the data is stored correctly
# open the saved file and check the first entries
# for serialized_example in tf.python_io.tf_record_iterator('test.tfrecords'):
#     example = tf.train.Example()
#     example.ParseFromString(serialized_example)
#     x_1 = np.array(example.features.feature['X'].float_list.value)
#     y_1 = np.array(example.features.feature['Y'].float_list.value)
#     break

# np_to_tfrecords(X, Y, file_path_prefix, verbose=True)

Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_10h_norm2\filenum0.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_10h_norm2\filenum0.tfrecords done!
Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_10h_norm2\filenum1.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_10h_norm2\filenum1.tfrecords done!
Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_10h_norm2\filenum2.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_10h_norm2\filenum2.tfrecords done!
Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_1

In [ ]:
for serialized_example in tf.python_io.tf_record_iterator('tfrecord_files/test3.tfrecords'):
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    x_1 = np.array(example.features.feature['X'].float_list.value)
    y_1 = np.array(example.features.feature['Y'].float_list.value)
    break

In [84]:
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/10hdata'
input_name = 'ftr_refrmd_norm2'
input_path = os.path.normpath(os.path.join(write_path,input_name))
target_name = 'single_dataset_log_10h_norm'
target_path = os.path.normpath(os.path.join(write_path,target_name))
h5f1 = h5py.File(input_path+'.hdf5','r')
hh = h5py.File(target_path+'.hdf5', 'r')
d=hh[target_name]
data_len = d.shape[0]
x = h5f1[input_name]
x.shape

(2204556, 771)

In [52]:
mixed = h5f1[normalized_file][2000000:]
mixed

array([[ 0.266125  ,  0.1387287 , -0.71211874, ..., -1.601058  ,
        -1.10846   , -0.46532393],
       [ 0.5477439 ,  0.24598686, -1.4461049 , ..., -0.6047218 ,
        -0.48930135, -0.44956982],
       [-0.06976136, -0.07189065, -0.5253663 , ..., -1.0566396 ,
        -1.2633575 , -1.1670849 ],
       ...,
       [-0.10297483, -0.31201565, -0.76192707, ..., -1.1310498 ,
        -1.169964  , -1.3364681 ],
       [-0.7514766 , -0.39640373, -0.7915892 , ..., -0.99282587,
        -1.0084419 , -1.2564884 ],
       [-0.89049244, -0.2596623 , -0.49888033, ...,  1.9272414 ,
         1.9187233 ,  1.9395918 ]], dtype=float32)

In [ ]:
#define reconstruct function to reconstruct sound from framed signal.
def reconstruct(wave,angle):
    recon = np.sqrt(np.power(10, wave))
    recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=256, win_length=512, window='hann')
    return recon

In [95]:
hh.close()
# f.close()
h5f1.close()

In [33]:
tot_len

2197278

In [ ]:
import h5py
import numpy as np
import librosa
h5f = h5py.File('mixed_log2.hdf5','r')
log = h5f['mixed_log2'][105:209]
h5f.close()
h5f = h5py.File('mixed_phase2.hdf5','r')
phase = h5f['mixed_phase2'][105:209]
h5f.close()
recon_mixed=reconstruct(log, phase)

In [ ]:
import librosa
h5f = h5py.File('single_dataset_log_16.hdf5','r')
log_single = h5f['single_dataset_log_16'][0:1000]
h5f.close()
h5f = h5py.File('single_dataset_phase_16.hdf5','r')
phase_single = h5f['single_dataset_phase_16'][0:1000]
h5f.close()
recon_single=reconstruct(log_single, phase_single)

In [ ]:
import sounddevice as sd
sd.play(recon_single,16000)

In [ ]:
log_single[0]

In [ ]:
import h5py
h5f = h5py.File('mixed_log2.hdf5','r')
log = h5f['mixed_log2'][0:1000]
h5f.close()
h5ff = h5py.File('ftr_refrmd.hdf5','r')
log3 = h5ff['ftr_refrmd'][0:1000]
h5ff.close()

In [ ]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [ ]:
h5f = h5py.File('mixed_log2.hdf5','r')
mix = h5f['mixed_log2'][0:]

In [ ]:
log3[291,257*4:257*5]

In [ ]:
import math as m
import numpy as np
import h5py
w = 7
import h5py
h5f = h5py.File('mixed_log2.hdf5','r')
ftr = h5f['mixed_log2'][0:]
h5f.close()
ftr_refrmd=[]
for i in range(m.floor(w/2)):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        temp.extend(ftr[i+p+j])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
f.close()
f = h5py.File('ftr_refrmd.hdf5', 'w')
d = f.create_dataset('ftr_refrmd', (0,257*7),maxshape=(None,257*7), dtype='f', chunks=True)
for i in range(m.floor(w/2),ftr.shape[0]-m.floor(w/2)-1):
    k=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        temp.extend(ftr[i+k])
        k=k+1
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
    if len(ftr_refrmd)>100000 or i==ftr.shape[0]-m.floor(w/2)-2:
        d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
        d[-1*len(ftr_refrmd):] = ftr_refrmd
        ftr_refrmd=[]
    if i%1000==0:
        print(i)
for i in range(ftr.shape[0]-m.floor(w/2)-1,ftr.shape[0]):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        if j>m.floor(w/2)+(ftr.shape[0]-i)-1:
            temp.extend(np.zeros(ftr[ftr.shape[0]-1].shape))
        else:
            temp.extend(ftr[i+p+j])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
d[-1*len(ftr_refrmd):] = ftr_refrmd
ftr_refrmd=[]
##############################################
##############################################


In [48]:
# h5.close()
# h5f.close()
f.close()


In [ ]:
import h5py
import numpy as np
h5 = h5py.File('ftr_refrmd.hdf5', 'r')
d=h5['ftr_refrmd']
tot_len=d.shape[0]
lst=0
h5.close()
on=0
h5f = h5py.File('ftr_refrmd.hdf5','r')
for i in range(0,tot_len,1000):
    mixed = h5f['ftr_refrmd'][i:i+1000]
    lst=lst+np.sum(mixed,axis=0)
    on=on+len(mixed)
tot_mean=lst/tot_len
h5f.close()
lst=0
h5f = h5py.File('ftr_refrmd.hdf5','r')
for i in range(0,tot_len,1000):
    mixed = h5f['ftr_refrmd'][i:i+1000]
    lst=lst+np.sum((mixed - tot_mean)**2,axis=0)
lst=lst/tot_len
tot_std=np.sqrt(lst)
h5f.close()


In [ ]:
f = h5py.File('ftr_scaled.hdf5', 'w')
dd = f.create_dataset('ftr_scaled', (0,1799),maxshape=(None,1799), dtype='f', chunks=True)
h5f = h5py.File('ftr_refrmd.hdf5','r')
for i in range(0,tot_len,1000):
    mixed = h5f['ftr_refrmd'][i:i+1000]
    scaled=(mixed-tot_mean)/tot_std
    dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
    dd[-1*scaled.shape[0]:] = scaled

In [ ]:
np.savetxt('mean.txt',tot_mean)
np.savetxt('std.txt',tot_std)

In [ ]:
import h5py
import numpy as np
h5 = h5py.File('ftr_refrmd.hdf5', 'r')
d=h5['ftr_refrmd']
tot_len=d.shape[0]
lst=0
h5.close()
on=0
h5f = h5py.File('ftr_refrmd.hdf5','r')
for i in range(0,tot_len,1000):
    mixed = h5f['ftr_refrmd'][i:i+1000]
    lst=lst+np.sum(mixed,axis=0)
    on=on+len(mixed)
tot_mean=lst/tot_len
h5f.close()
lst=0
h5f = h5py.File('ftr_refrmd.hdf5','r')
for i in range(0,tot_len,1000):
    mixed = h5f['ftr_refrmd'][i:i+1000]
    lst=lst+np.sum(mixed - tot_mean**2,axis=0)
lst=lst/tot_len
tot_std=np.sqrt(lst)
h5f.close()


In [ ]:
f.close()
f = h5py.File('ftr_scaled.hdf5', 'w')
dd = f.create_dataset('ftr_scaled', (0,1799),maxshape=(None,1799), dtype='f', chunks=True)
h5f = h5py.File('ftr_refrmd.hdf5','r')
for i in range(0,tot_len,1000):
    mixed = h5f['ftr_refrmd'][i:i+1000]
    scaled=(mixed-tot_mean)/tot_std
    dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
    dd[-1*scaled.shape[0]:] = scaled

In [ ]:
on

In [ ]:
h5f = h5py.File('ftr_refrmd.hdf5','r')
mixed = h5f['ftr_refrmd'][0]
h5f.close()
np.mean(mixed-tot_mean)

In [ ]:
import h5py
h5f = h5py.File('mixed_log2.hdf5','r')
mixed = h5f['mixed_log2'][0:110]
h5f.close()
print('loaded')
from sklearn import preprocessing
import numpy as np
scaler = preprocessing.StandardScaler().fit(mixed.T)
from sklearn.externals import joblib
scaler_filename = "standard_scaler"
joblib.dump(scaler, scaler_filename) 
#inverse_transform(X, copy=None)

In [ ]:
# And now to load...
from sklearn.externals import joblib
scaler = joblib.load('standard_scaler')
import h5py
h5f = h5py.File('mixed_log2.hdf5','r')
mixed = h5f['mixed_log2'][0:1000]
h5f.close()
print('loaded')
scaled_mix=scaler.transform(mixed.T)


In [ ]:
import h5py
h5f = h5py.File('scaled_mix.h5', 'w')
h5f.create_dataset('scaled_mix', data=scaled_mix.T)
h5f.close()


In [ ]:
f.close()

In [ ]:
import h5py
import math as m
import os
import librosa
import numpy as np
orig_path=os.getcwd()
wav=[]
l=os.listdir(os.path.join(orig_path,'single_dataset_16'))
l.sort(key=int)
filename1 = l[1]
i=0
w=7
num_of_snrs=21
num_of_speakers=6
files = os.listdir(os.path.join(orig_path,'single_dataset_16',filename1))
f.close()
f = h5py.File('single_dataset_log_16.hdf5', 'w')
d = f.create_dataset('single_dataset_log_16', (0,257),maxshape=(None,257), dtype='f', chunks=True)
#in this section the features for clean voices are extracted.
for voice in files:
    #sr, wav_data = wavfile.read(os.path.join(orig_path,'single dataset',filename1,filename2)+"\\"+voice, )
    #sr, wav_data = wavfile.read(os.path.join(orig_path,'mixed2_with24')+"\\"+filename)
    wav_data, sr = librosa.load(os.path.join(orig_path,'single_dataset_16',filename1,voice), sr=16000) 
    framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
    #frame length = win_length * freq *1000(ms)
    abslt=np.absolute(framed_data)**2
    dft_signal=np.log10(abslt+1e-7*np.ones(np.shape(abslt)))
#     for i in range(m.floor(w/2)):
#         dft_signal=np.insert(dft_signal,0,0,axis = 1)
    #data_phase=np.angle(framed_data)
    for i in range(num_of_snrs*num_of_speakers):
        d.resize(d.shape[0]+dft_signal.shape[1], axis=0)   
        d[-1*dft_signal.shape[1]:] = dft_signal.T
    #np.savetxt('single_dataset_log_16\\abs'+'2'+'_'+voice+'.txt', dft_signal, delimiter=',')
    #np.savetxt('single_dataset_phase_features_256\\phase'+'1'+'_'+str(i+1)+'.txt', data_phase, delimiter=',')
f.close()

    


In [ ]:
import h5py
h5f = h5py.File('ftr_refrmd.hdf5','r')
single = h5f['ftr_refrmd'][-1]
h5f.close()


In [ ]:
import h5py
h5f = h5py.File('single_dataset_log_16.hdf5','r')
single = h5f['single_dataset_log_16'][:]
h5f.close()


In [ ]:
single.shape[0]

In [ ]:
del scaled_mix

In [ ]:
import h5py
h5f = h5py.File('mixed_log2.hdf5','r')
ftr = h5f['mixed_log2'][0:]
h5f.close()

In [ ]:
import numpy as np
ftr_refrmd = np.zeros((983134,257*7))

In [ ]:
ftr.shape

In [ ]:
import math as m
import numpy as np
import h5py
w = 7
ftr_refrmd=[]
for i in range(m.floor(w/2)):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        #if j<m.floor(w/2)-i+1:
            #temp.extend(np.zeros(ftr[0].shape))
        #else:
        #if j>m.floor(w/2)-i+1:
        temp.extend(ftr[i+p+j])
        #temp.extend(ftr[j-(m.floor(w/2)-i+1)])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
f.close()
f = h5py.File('ftr_refrmd.hdf5', 'w')
d = f.create_dataset('ftr_refrmd', (0,257*7),maxshape=(None,257*7), dtype='f', chunks=True)
for i in range(m.floor(w/2),ftr.shape[0]-m.floor(w/2)-1):
    k=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        temp.extend(ftr[i+k])
        k=k+1
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
    if len(ftr_refrmd)>100000 or i==ftr.shape[0]-m.floor(w/2)-2:
        d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
        d[-1*len(ftr_refrmd):] = ftr_refrmd
        ftr_refrmd=[]
    if i%1000==0:
        print(i)
for i in range(ftr.shape[0]-m.floor(w/2)-1,ftr.shape[0]):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        if j>m.floor(w/2)+(ftr.shape[0]-i)-1:
            temp.extend(np.zeros(ftr[ftr.shape[0]-1].shape))
        else:
            temp.extend(ftr[i+p+j])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
d[-1*len(ftr_refrmd):] = ftr_refrmd
ftr_refrmd=[]

In [ ]:
ftr[6]

In [ ]:
ftr_refrmd[0][-257:]

In [ ]:
import h5py
h5f = h5py.File('single_dataset_log_16.hdf5','r')
single_dataset = h5f['single_dataset_log_16'][0:]
h5f.close()

In [ ]:
single_dataset.shape

In [ ]:
 np.arange(0,21*6*,2)

In [ ]:
orig_path=os.getcwd()
from natsort import natsorted
# import h5py
# h5f = h5py.File('single_dataset_log_16.hdf5','r')
# single_dataset = h5f['single_dataset_log_16'][0:]
# h5f.close()
target1 = single_dataset[0].T 
for i in 
target = single_dataset[0].T
#target=np.loadtxt('single_dataset_log_16\\'+'abs1_1.txt',delimiter=',').T
#target1=np.loadtxt('single_dataset_log_16\\'+'abs1_1.txt',delimiter=',').T
for i in range(21*6):
    target1=np.concatenate((target1, target),axis=0)
print(target1.shape)
j=1
a=os.listdir(os.path.join(orig_path,'single_dataset_log_16'))
a=natsorted(a)
for filename in a[1:]:
    f=np.loadtxt('single_dataset_log_16\\'+filename,delimiter=',')
    print(f.shape)
    for i in range(600):
        target1=np.concatenate((target1,f.T),axis=0)
    #print(j)
    print(target1.shape)
    if j==4:
        break
    j=j+1
print(target1.shape)

In [ ]:
import numpy as np
import os
from scipy.io import wavfile
import librosa
import csv
import math as m
from natsort import natsorted
import h5py
#in this section the features for mixed voices are extracted.
orig_path=os.getcwd()
i = 1
a = os.listdir(os.path.join(orig_path,'sec_mixed2_6'))
a = natsorted(a)

In [ ]:
a